In [ ]:
from google.colab import files
uploaded = files.upload()


Saving 100points.csv to 100points (4).csv


In [ ]:
import pandas as pd
import io
data1 = pd.read_csv(io.BytesIO(uploaded['100points.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
!pip install --pre pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pycaret.datasets import get_data
data1 
data = data1.dropna()
data


,date,totalusage
0,10/12/2015,0.000204
1,10/12/2015,0.000240
2,10/12/2015,0.000236
3,10/12/2015,0.000240
4,10/12/2015,0.000240
...,...,...
94,14/12/2015,2.764792
95,14/12/2015,1.713692
96,14/12/2015,0.927507
97,14/12/2015,2.237864


In [ ]:
!pip uninstall matplotlib
!pip install matplotlib==3.1.3
import matplotlib.pyplot as plt


Found existing installation: matplotlib 3.5.3
Uninstalling matplotlib-3.5.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.5.3-py3.7-nspkg.pth
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.5.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/matplotlib/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/tests/*
    /usr/local/lib/python3.7/dist-packages/pylab.py
Proceed (y/n)? y
  Successfully uninstalled matplotlib-3.5.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 4.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

In [ ]:
data.dtypes
data['date'] = pd.to_datetime(data['date'])
# create 30 day  moving average
data['MA30'] = data['totalusage'].rolling(30).mean()

# plot the data and MA
import plotly.express as px
fig = px.line(data, x="date", y=["totalusage", "MA30"], template = 'plotly_dark')
fig.show()

In [ ]:

# extract month and year from dates
import numpy as np

# extract features from date**
data['month'] = [i.month for i in data['date']]
data['year'] = [i.year for i in data['date']]
# data['day_of_week'] = [i.dayofweek for i in data['date']]
data['day'] = [i.day for i in data['date']]
# data['day_of_year'] = [i.dayofyear for i in data['date']]


data['Series'] = np.arange(1,len(data)+1)
#drop unnecessary columns and re-arrange
data.drop([ 'MA30'], axis=1, inplace=True)
# check the head of the dataset

data = data[['Series','date','totalusage','day','month','year']]
# 'day_of_year''day_of_week',
data.head()

,Series,date,totalusage,day,month,year
0,1,2015-10-12,0.000204,12,10,2015
1,2,2015-10-12,0.000240,12,10,2015
2,3,2015-10-12,0.000236,12,10,2015
3,4,2015-10-12,0.000240,12,10,2015
4,5,2015-10-12,0.000240,12,10,2015


In [ ]:
print(data.isnull().sum())
data['totalusage'].round(decimals = 5)
data.info()

Series        0
date          0
totalusage    0
day           0
month         0
year          0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 98
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Series      99 non-null     int64         
 1   date        99 non-null     datetime64[ns]
 2   totalusage  99 non-null     float64       
 3   day         99 non-null     int64         
 4   month       99 non-null     int64         
 5   year        99 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 5.4 KB


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# import datetime 
# split_date = datetime.date(2015,12,13)
# train = data[(pd.to_datetime(data["date"]).dt.date < split_date)]
# test = data[(pd.to_datetime(data["date"]).dt.date > split_date)]
# train.shape, test.shape

((59, 6), (16, 6))

In [ ]:
# import the regression module
from pycaret.regression import *
# initialize setup
s = setup( data, target = 'totalusage', fold_strategy = 'timeseries', fold = 3, session_id = 123)



,Description,Value
0,Session id,123
1,Target,totalusage
2,Target type,Regression
3,Data shape,"(99, 6)"
4,Train data shape,"(69, 6)"
5,Test data shape,"(30, 6)"
6,Numeric features,4
7,Date features,1
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best = compare_models(sort = 'MAE',errors="raise")


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.7722,1.0378,1.0173,0.4488,0.3523,37.1300,0.1033
ada,AdaBoost Regressor,0.8690,1.6024,1.2617,0.1492,0.3977,0.7407,0.0600
rf,Random Forest Regressor,0.8856,1.5349,1.2293,0.1776,0.3943,0.7493,0.2667
et,Extra Trees Regressor,0.9693,2.0506,1.4116,-0.1221,0.4297,20.8804,0.2067
en,Elastic Net,0.9833,1.5030,1.2206,0.1753,0.4265,423.5595,0.0367
lasso,Lasso Regression,0.9845,1.4992,1.2193,0.1785,0.4276,439.5146,0.0400
br,Bayesian Ridge,0.9902,1.5073,1.2230,0.1751,0.4313,459.0557,0.0400
gbr,Gradient Boosting Regressor,1.0007,2.0343,1.4026,-0.1355,0.4416,7.7878,0.0567
ridge,Ridge Regression,1.0213,1.5806,1.2548,0.1340,0.4433,461.1207,0.0400
dt,Decision Tree Regressor,1.0376,2.2082,1.4618,-0.2432,0.4656,0.8801,0.0367


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [ ]:
prediction_holdout = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,K Neighbors Regressor,0.9641,1.7480,1.3221,0.3830,0.4030,0.5912


In [ ]:
# generate predictions on the original dataset
predictions = predict_model(best, data=data)
# line plot

predictions['Date']= data['date']

fig = px.line(predictions, x='Date', y=["totalusage", "prediction_label"], template = 'plotly_dark')
# add a vertical rectange for test-set separation
# fig.add_vrect(x0="-01-01", x1="2015-12-13", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,K Neighbors Regressor,0.7142,1.0526,1.0259,0.5644,0.3227,14.8543


In [ ]:
predictions

,Series,date_month,date_day,day,month,totalusage,prediction_label,Date
0,1.0,10.0,12.0,12.0,10.0,0.000204,0.000245,2015-10-12
1,2.0,10.0,12.0,12.0,10.0,0.000240,0.000245,2015-10-12
2,3.0,10.0,12.0,12.0,10.0,0.000236,0.000245,2015-10-12
3,4.0,10.0,12.0,12.0,10.0,0.000240,0.000245,2015-10-12
4,5.0,10.0,12.0,12.0,10.0,0.000240,0.000245,2015-10-12
...,...,...,...,...,...,...,...,...
94,95.0,12.0,14.0,14.0,12.0,2.764792,2.508042,2015-12-14
95,96.0,12.0,14.0,14.0,12.0,1.713692,2.459061,2015-12-14
96,97.0,12.0,14.0,14.0,12.0,0.927507,2.005352,2015-12-14
97,98.0,12.0,14.0,14.0,12.0,2.237864,2.005352,2015-12-14


In [ ]:
final_best = finalize_model(best)

In [ ]:
future_dates = pd.date_range(start = '14/12/2015', end = '14/02/2016', freq = 'MS')
future_df = pd.DataFrame()
future_df['date'] = pd.date_range(start = '14/12/2015', end = '14/02/2016', freq = 'MS')
future_df['month'] = [i.month for i in future_dates]
future_df['year'] = [i.year for i in future_dates]    
future_df['day'] = [i.day for i in future_dates]
future_df['Series'] = np.arange(100,(100+len(future_dates)))
future_df.head()

,date,month,year,day,Series
0,2016-01-01,1,2016,1,100
1,2016-02-01,2,2016,1,101


In [ ]:
data = data[['Series','date','totalusage','day','month','year']]

# predictions_future.rename(columns = {'prediction_label':'totalusage'}, inplace = True)
predictions_future.drop([ 'date_month'])
predictions_future.drop([ 'date_day'])
predictions_future['year'] = future_df['year']
predictions_future['date'] = future_df['date']

predictions_future = predictions_future[['Series','date','totalusage','day','month','year']]
predictions_future


KeyError: ignored

In [ ]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future

,Series,date_month,date_day,day,month,prediction_label
0,100.0,1.0,1.0,1.0,1.0,2.005352
1,101.0,2.0,1.0,1.0,2.0,2.005352


In [ ]:
concat_df = pd.concat([data,predictions_future], axis=0)
concat_df_i = pd.date_range(start='10/12/2015',  end = '14/02/2016', freq = 'MS')
fig = px.line(concat_df, x="date", y=["totalusage"], template = 'plotly_dark')
fig.show()

In [ ]:
concat_df

,Series,date,totalusage,day,month,year
0,1.0,2015-10-12,0.000204,12.0,10.0,2015
1,2.0,2015-10-12,0.000240,12.0,10.0,2015
2,3.0,2015-10-12,0.000236,12.0,10.0,2015
3,4.0,2015-10-12,0.000240,12.0,10.0,2015
4,5.0,2015-10-12,0.000240,12.0,10.0,2015
...,...,...,...,...,...,...
96,97.0,2015-12-14,0.927507,14.0,12.0,2015
97,98.0,2015-12-14,2.237864,14.0,12.0,2015
98,99.0,2015-12-14,2.382909,14.0,12.0,2015
0,100.0,2016-01-01,2.005352,1.0,1.0,2016


In [ ]:
''
